Import Necessary Packages

In [4]:
import pandas as pd
import numpy as np
import re
import math
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support
from scipy import sparse
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

Preprocess Data

In [5]:
def formatTweets(fileLocation):
    dataFrameVals = pd.read_excel(fileLocation)
    p = PorterStemmer()
    stopWords = set(stopwords.words("english"))
    tweetList=dataFrameVals['Tweet']
    classList=dataFrameVals['Class']
    formattedTweets = []
    classes = []
    for record,label in zip(tweetList,classList):
        if(isinstance(label, str) != True):
            if (label !=2 and math.isnan(label) !=True):
                regexHTMLRemove = re.compile(r'<.*?>')
                tempString = regexHTMLRemove.sub('',str(record)).lower()             
                #Replace all URLS
                tempString = re.sub(r'(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))',' ',tempString)
                #get all A to Z
                tempString = re.sub(r"[^A-Za-z ]+", ' ', tempString) 
                listOfWords = tempString.split()
                newList = []
                for word in listOfWords:
                    if word not in stopWords:
                        newList.append(p.stem(word))
                tempString = " ".join(str(x) for x in newList)              
                formattedTweets.append(tempString)
                classes.append(label)
    return formattedTweets,classes

Enter File Location

In [6]:
fileLocation = <FileLocation>tweets,classLabels=formatTweets(fileLocation)

Format Data to feed the model

In [7]:
countVector = CountVectorizer()
XTrainCount = countVector.fit_transform(tweets)
XTrainTF = TfidfTransformer(use_idf=False).fit(XTrainCount).transform(XTrainCount)
tfidf_transformer = TfidfTransformer()
XTrainTFIDF = tfidf_transformer.fit_transform(XTrainCount)

Split into 10 Folds and evaluate

In [9]:
kf = KFold(n_splits=10)
sumf_1=0
sumf1=0
sumprec1=0
sumprec_1=0
sumrec1=0
sumrec_1=0
avgerror=0
for train,test in kf.split(XTrainTFIDF):
    predicted=[]
    trainset=[]
    testset=[]
    truevalues=[]
    trainlabels=[]
    sumerror=0
    for i in train:
        trainset.append(XTrainTFIDF[i].toarray()[0])
        trainlabels.append(classLabels[i])
    trainmatrix=sparse.csc_matrix(np.asarray(trainset))
    for i in test:
        testset.append(XTrainTFIDF[i].toarray()[0])
        truevalues.append(classLabels[i])
    testmatrix=sparse.csc_matrix(np.asarray(testset))
    knn = KNeighborsClassifier(n_neighbors=6)
    knn.fit(trainset, trainlabels) 
    predicted = knn.predict(np.asarray(testset))
    for i in range(len(predicted)):
        if(predicted[i]!=truevalues[i]):
            sumerror=sumerror+1
    avgerror=avgerror+float(sumerror)/len(test)
    sumprec_1=sumprec_1+precision_recall_fscore_support(predicted,truevalues,labels=[-1,1])[0][0]
    sumprec1=sumprec1+precision_recall_fscore_support(predicted,truevalues,labels=[-1,1])[0][1]
    sumrec_1=sumrec_1+precision_recall_fscore_support(predicted,truevalues,labels=[-1,1])[1][0]
    sumrec1=sumrec1+precision_recall_fscore_support(predicted,truevalues,labels=[-1,1])[1][1]
    sumf_1=sumf_1+precision_recall_fscore_support(predicted,truevalues,labels=[-1,1])[2][0]
    sumf1=sumf1+precision_recall_fscore_support(predicted,truevalues,labels=[-1,1])[2][1]
print("Average Precision for Class 1",sumprec1/10)
print("Average Recall for Class 1",sumrec1/10)
print("Average Precision for Class -1",sumprec_1/10)
print("Average Recall for Class -1",sumrec_1/10)
print("Average FScore for Class 1",sumf1/10)
print("Average FScore for Class -1",sumf_1/10)
print("Average Error",avgerror/10)

Average Precision for Class 1 0.262760754469
Average Recall for Class 1 0.468956520889
Average Precision for Class -1 0.832503765158
Average Recall for Class -1 0.571728634657
Average FScore for Class 1 0.328301422053
Average FScore for Class -1 0.663501442457
Average Error 0.46705579614636294
